### 1. combine `mon_features.pkl` & `unmon_features.pkl` into `features_df`

In [1]:
import pandas as pd
import numpy as np
import pickle

MON_FILE_PATH = '/content/mon_features.pkl'
UNMON_FILE_PATH = '/content/unmon_features.pkl'

LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER2 = [
    'num_outgoing_packets', 'incoming_packet_ratio',
    'outgoing_order_skew', 'outgoing_first_30',
    'avg_outgoing_order_first_30', 'avg_incoming_burst_size',
    'cumul_packets_10pct', 'cumul_packets_30pct',
    'cumul_max'
]

mon_features_df = pd.read_pickle(MON_FILE_PATH)
unmon_features_df = pd.read_pickle(UNMON_FILE_PATH)

features_df = pd.concat([mon_features_df, unmon_features_df], ignore_index=True)

X = features_df[FEATURES_VER2]
y = features_df[LABEL_COLUMN[0]]

print(X)
print(y)

       num_outgoing_packets  incoming_packet_ratio  outgoing_order_skew  \
0                     121.0               0.914849            -0.257072   
1                      80.0               0.845560             0.153926   
2                     118.0               0.913108            -0.463423   
3                     122.0               0.915629            -0.391122   
4                     115.0               0.918208            -0.355596   
...                     ...                    ...                  ...   
28995                 413.0               0.910081            -0.272106   
28996                 447.0               0.912524            -0.550236   
28997                  59.0               0.836565            -0.028954   
28998                  96.0               0.811395            -0.703690   
28999                 322.0               0.967768             0.045687   

       outgoing_first_30  avg_outgoing_order_first_30  \
0               0.300000                  

### 2. SVM

Ignore ConvergenceWarning

In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

Split the dataset into training and testing sets

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Map all points to have mean=0 and std=1

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train.values)
X_test_scale = scaler.transform(X_test.values)
X_scale = scaler.fit_transform(X.values)

Train and test SVM using RBF kernel

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

clf_rbf = SVC(kernel='rbf', C=100, gamma=0.1, class_weight='balanced')
clf_rbf.fit(X_train_scale, y_train)
y_pred_rbf = clf_rbf.predict(X_test_scale)

print("============ Before Hyperparameter tuning ============")
print("SVM Accuracy: {}".format(accuracy_score(y_test, y_pred_rbf)))
print("Classification Report:\n", classification_report(y_test, y_pred_rbf))

============ Before Hyperparameter tuning ============
SVM Accuracy: 0.7666206896551724
Classification Report:
               precision    recall  f1-score   support

          -1       0.98      0.75      0.85      2470
           0       0.63      0.79      0.70        42
           1       0.80      0.84      0.82        51
           2       0.36      0.94      0.52        47
           3       0.57      0.90      0.70        51
           4       0.78      0.84      0.81        43
           5       0.78      0.82      0.80        44
           6       0.77      0.91      0.83        53
           7       0.63      0.65      0.64        55
           8       0.67      0.51      0.58        47
           9       0.65      0.58      0.61        45
          10       0.74      0.73      0.74        44
          11       0.65      0.82      0.73        55
          12       0.70      0.94      0.80        52
          13       0.42      0.37      0.39        57
          14       0.65

### 3. Hyperparameter tuning by using Grid Search

In [6]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

search = HalvingGridSearchCV(SVC(class_weight='balanced'), param_grid, cv=5, factor=3, n_jobs=-1)
search.fit(X_train_scale, y_train)
y_pred_tuned = search.predict(X_test_scale)

print("============ After Hyperparameter tuning ============")
print("SVM Accuracy: {}".format(accuracy_score(y_test, y_pred_tuned)))
print("Classification Report:\n", classification_report(y_test, y_pred_tuned))

============ After Hyperparameter tuning ============
SVM Accuracy: 0.7990344827586207
Classification Report:
               precision    recall  f1-score   support

          -1       0.90      0.90      0.90      2470
           0       0.78      0.74      0.76        42
           1       0.85      0.80      0.83        51
           2       0.53      0.85      0.65        47
           3       0.68      0.86      0.76        51
           4       0.61      0.81      0.70        43
           5       0.80      0.80      0.80        44
           6       0.84      0.87      0.85        53
           7       0.60      0.49      0.54        55
           8       0.68      0.60      0.64        47
           9       0.61      0.67      0.64        45
          10       0.79      0.70      0.75        44
          11       0.78      0.71      0.74        55
          12       0.90      0.85      0.87        52
          13       0.48      0.51      0.50        57
          14       0.68 